In [11]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.pardir))
sys.path.append(parent_dir)

from libsql_graph_db import database as db
from dotenv import load_dotenv # type: ignore

load_dotenv()
db_url = os.getenv("LIBSQL_URL")
auth_token = os.getenv("LIBSQL_AUTH_TOKEN")

In [12]:
# To initialze the database and establish a connection with tursodb
db.initialize(db_url, auth_token)

thread '<unnamed>' panicked at /root/.cargo/git/checkouts/libsql-311658d335deb3b1/9de3ccc/libsql/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace
Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/libsql_graph_db/database.py", line 59, in initialize
    return atomic(db_url, auth_token, _init)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [13]:
# To add a single node
def insert_single_node(db_url, auth_token, new_node, new_node_id):
    try:
        db.atomic(db_url, auth_token, db.add_node(new_node, new_node_id))
    except Exception as e:
        assert False
new_node = {'subject': 'Civil', 'type': ['person', 'loves physics']}
new_node_id = 'Student'

insert_single_node(db_url, auth_token, new_node, new_node_id)
    

In [3]:
# To add multiple nodes
def bulk_insert_operations(db_url, auth_token, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    # bulk add and confirm
    db.atomic(db_url, auth_token, db.add_nodes(bodies, ids))

new_nodes = {
    'Student': {'name': 'Stanley', 'age': '20'},
    'Principal': {'name': 'Jenny', 'age': '55'}
}

bulk_insert_operations(db_url, auth_token, new_nodes)

In [6]:
# To find for a node
def find_a_node(db_url, auth_token, node):
            return db.atomic(db_url, auth_token, db.find_node(node))


find_a_node(db_url, auth_token, 'Bill Gates')

Traceback (most recent call last):
  File "/tmp/ipykernel_107091/1153528586.py", line 3, in find_a_node
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
thread '<unnamed>' panicked at /root/.cargo/git/checkouts/libsql-311658d335deb3b1/9de3ccc/libsql/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime


{'name': 'Bill Gates',
 'toy_name': 'soft toy',
 'animal': 'lion',
 'id': 'Bill Gates'}

In [10]:
# To find multiple nodes
def find_bulk_node(db_url, auth_token, node):
        return db.atomic(db_url, auth_token, db.find_node(node))



find_bulk_node(db_url, auth_token, 'Student')

Traceback (most recent call last):
  File "/tmp/ipykernel_107091/3974166414.py", line 3, in find_bulk_node
thread '<unnamed>' panicked at /root/.cargo/git/checkouts/libsql-311658d335deb3b1/9de3ccc/libsql/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


{'name': 'Stanley', 'age': '20', 'id': 'Student'}

In [12]:
# For bulk upsert: Update and insert
def bulk_upsert(db_url, auth_token, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    db.atomic(db_url, auth_token, db.upsert_nodes(bodies, ids))
nodes = {
    'Student': {'name': 'Stanley', 'age': '30'},
    'Professor': {'name': 'James', 'age': '35'}
}

bulk_upsert(db_url, auth_token, nodes)
    

In [13]:
# For single upsert: Update and insert a node
def single_upsert(db_url, auth_token, body, id):
    print(body, id)
    
    db.atomic(db_url, auth_token, db.upsert_node(id, body))

body = {'name': 'Bob', 'age': '23'}
id = 'Student'
single_upsert(db_url, auth_token, body, id)

{'name': 'Bob', 'age': '23'} Student


In [14]:
# Bulk connect and confirm
def bulk_node_connect(db_url, auth_token, edges):
    sources = []
    targets = []
    properties = []
    for src, tgts in edges.items():
        for target in tgts:
            tgt, label = target
            sources.append(src)
            targets.append(tgt)
            if label:
                properties.append(label)
            else:
                properties.append({})
        

    db.atomic(db_url, auth_token, db.connect_many_nodes(
        sources, targets, properties))
    
edges = {
    'Professor': [('Bill Gates', {'wealth':'1000 Billion'})],
    'Bill Gates': [('Professor', {'balance': '1000 rupees'})]
}

bulk_node_connect(db_url, auth_token, edges)
    


In [15]:
# Single connect and confirm
def single_node_connect(db_url, auth_token, sources, target,  property):
    db.atomic(db_url, auth_token, db.connect_nodes(
        source, target,  property))
    
source = 'Engineer'
target = 'Bill Gates'
property = {'package': '100000 lpa'}

single_node_connect(db_url, auth_token, source, target, property)

In [16]:
# To remove nodes
def remove_bulk_nodes(db_url, auth_token, ids):
        db.atomic(db_url, auth_token,  db.remove_nodes(ids))

ids = ['Student']

remove_bulk_nodes(db_url, auth_token, ids)


In [4]:
# To remove a single node
def remove_single_node(db_url, auth_token, id):
    db.atomic(db_url, auth_token, db.remove_node(id))

remove_single_node(db_url, auth_token, 'Professor')

In [17]:
# To traverse the nodes
def traverse_nodes(db_url, auth_token, src, tgt=None):
    return db.traverse(db_url, auth_token, src, tgt)
traverse_nodes(db_url, auth_token, 'Engineer', 'Bill Gates')

Traceback (most recent call last):
  File "/home/anubhuti/Desktop/simple-graph-db/src/simple_graph_sqlite/database.py", line 234, in traverse
    return atomic(db_url, auth_token, _traverse)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
thread '<unnamed>' panicked at /root/.cargo/git/checkouts/libsql-311658d335deb3b1/9de3ccc/libsql/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime


Bill Gates
Here Engineer
Here Bill Gates
Here Professor


['Engineer', 'Bill Gates', 'Professor']

In [18]:
# To generate a query clause
kv_name_like = db._generate_clause('name', predicate='LIKE')
print(db.atomic(db_url, auth_token, db.find_nodes([kv_name_like], ('Bill%',))))


Traceback (most recent call last):
  File "/tmp/ipykernel_107091/1985963463.py", line 3, in <module>
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


[{'name': 'Bill Gates', 'toy_name': 'soft toy', 'animal': 'lion', 'id': 'Bill Gates'}]


thread '<unnamed>' panicked at /root/.cargo/git/checkouts/libsql-311658d335deb3b1/9de3ccc/libsql/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime


In [19]:
# Visualization
from libsql_graph_db import visualizers
raw_path = "temp_path" +  "sample.dot"
visualizers.graphviz_visualize(db_url, auth_token, raw_path, ['Engineer', 'Bill Gates'])

Traceback (most recent call last):
thread '<unnamed>' panicked at /root/.cargo/git/checkouts/libsql-311658d335deb3b1/9de3ccc/libsql/src/hrana/hyper.rs  File "/home/anubhuti/Desktop/simple-graph-db/src/simple_graph_sqlite/visualizers.py", line 37, in graphviz_visualize
    for edge in db.atomic(db_url, auth_token, connections(i)):
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime
:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
Traceback (most recent call last):
thread '<unnamed>' panicked at   File "/home/anubhuti/Desktop/simple-graph-db/src/simple_graph_sqlite/visualizers.py", line 37, in graphviz_visualize
/root/.cargo/git/checkouts/libsql-311658d335deb3b1/9de3ccc/libsql/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    for edge in db.atomic(db_url, auth_tok

In [20]:
# Visualization with bodies

raw_path = "temp_path" +  "bodies.dot"
path_with_bodies = db.traverse(db_url, auth_token, 'Engineer', 'Bill Gates', with_bodies=True)
visualizers.graphviz_visualize_bodies(raw_path, path_with_bodies)

Traceback (most recent call last):
thread '<unnamed>  File "/home/anubhuti/Desktop/simple-graph-db/src/simple_graph_sqlite/database.py", line 234, in traverse
' panicked at /root/.cargo/git/checkouts/libsql-311658d335deb3b1/9de3ccc/libsql/src/hrana/hyper.rs:88:9:
there is no reactor running, must be called from the context of a Tokio 1.x runtime
    return atomic(db_url, auth_token, _traverse)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
pyo3_runtime.PanicException: there is no reactor running, must be called from the context of a Tokio 1.x runtime


Bill Gates
Here Engineer
Here Bill Gates
Here Bill Gates
Here Bill Gates
Here Professor
Here Engineer
Here Engineer
Here Professor
Here Professor
Here Bill Gates
Here Bill Gates
